# Brooklyn DEP

Remember that I've processed the following data sets from DEP (in data/processed/DEP/):

  1.  2021-311.parq
  2.  ms4-drainage.parq
  3.  ms4-outfalls.parq
  4.  extreme-flood-map.parq
  5.  moderate-flood.parq
  6.  Catch basins
  
I will use the Brooklyn bourough boundary and spatial joins to get the data within Brooklyn.

**Note:** Maybe I want to do another data pull process and include 2021-now?  Just a thought.

In [ ]:
brooklyn_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-boundary.parq')

# DEP 311

In [ ]:
dep_2021_311_gdf = gpd.read_parquet('../data/processed/DEP/2021-311.parq')

In [ ]:
brooklyn_2021_311_gdf = dep_2021_311_gdf.set_crs('epsg:4326').sjoin(brooklyn_gdf.to_crs('epsg:4326'), how='inner', predicate='within')

As is usual we can explore the data set a bit.

In [ ]:
len(brooklyn_2021_311_gdf)

In [ ]:
len(dep_2021_311_gdf)

In [ ]:
__ / _

Alright, so **27%** of the DEP 311 request calls in 2021 are in Brooklyn.  I suppose that is ok??

This is a lot of crap on a map.  I looked at it and it seems ok.  Going to comment it out for now.  Do it if you want.

In [ ]:
#brooklyn_2021_311_gdf[['Descriptor', 'geometry']].explore('Descriptor')

In [ ]:
brooklyn_2021_311_gdf['Complaint Type'].value_counts()

I'm ready to save this dataframe.

In [ ]:
brooklyn_2021_311_gdf.to_parquet('../data/processed/brooklyn/brooklyn-2021-311.parq')

# MS4

The two types of MS4 data I'm starting with:

  -  drainage
  -  outfalls

In [ ]:
ms4_drainage_gdf = gpd.read_parquet('../data/processed/DEP/ms4-drainage.parq')
ms4_outfalls_gdf = gpd.read_parquet('../data/processed/DEP/ms4-outfalls.parq')

Need to check the CRS for the spatial operations.

In [ ]:
brooklyn_gdf.crs

In [ ]:
ms4_drainage_gdf.crs

In [ ]:
ms4_outfalls_gdf.crs

Gonna do the ops with epsg=2263.

For each join I will do the op, check sizes and stuff, then visualize.

You'll see things make sense.

In [ ]:
brooklyn_ms4_outfalls_gdf = ms4_outfalls_gdf.sjoin(brooklyn_gdf.to_crs('epsg:2263'), how='inner', predicate='within')

In [ ]:
len(brooklyn_ms4_outfalls_gdf)

In [ ]:
brooklyn_ms4_outfalls_gdf.explore()

In [ ]:
brooklyn_ms4_drainage_gdf = brooklyn_gdf.to_crs('epsg:2263').overlay(ms4_drainage_gdf, how='intersection')

In [ ]:
len(brooklyn_ms4_drainage_gdf)

In [ ]:
brooklyn_ms4_drainage_gdf.explore()

Can save these two files now.

In [ ]:
brooklyn_ms4_drainage_gdf.to_parquet('../data/processed/brooklyn/brooklyn-ms4-drainage.parq')
brooklyn_ms4_outfalls_gdf.to_parquet('../data/processed/brooklyn/brooklyn-ms4-outfalls.parq')

# Flood Maps

DEP (with MOCR) published two different flood maps:

  -  Extreme flooding
  -  Moderate flooding
  
**Note:** Get references to who build the models and such.

In [ ]:
extreme_flood_gdf = gpd.read_parquet('../data/processed/DEP/extreme-flood-map.parq')
moderate_flood_gdf = gpd.read_parquet('../data/processed/DEP/moderate-flood-map.parq')

In [ ]:
extreme_flood_gdf.crs

In [ ]:
%%time
brooklyn_extreme_flood_gdf = brooklyn_gdf.to_crs('epsg:2263').overlay(extreme_flood_gdf, how='intersection')
brooklyn_moderate_flood_gdf = brooklyn_gdf.to_crs('epsg:2263').overlay(moderate_flood_gdf, how='intersection')

In [ ]:
brooklyn_extreme_flood_gdf.columns

In [ ]:
len(brooklyn_extreme_flood_gdf)

In [ ]:
brooklyn_extreme_flood_gdf.explore('desc')

In [ ]:
brooklyn_moderate_flood_gdf.explore('desc')

Once again, this seems to pass the goofy test.  Can save it.

In [ ]:
brooklyn_extreme_flood_gdf.to_parquet('../data/processed/brooklyn/brooklyn-extreme-flood.parq')
brooklyn_moderate_flood_gdf.to_parquet('../data/processed/brooklyn/brooklyn-moderate-flood.parq')

# Catch Basins

Late addition to the dataset.

In [ ]:
catch_basins_gdf = gpd.read_parquet('../data/processed/DEP/catch-basins.parq')

In [ ]:
len(catch_basins_gdf)

In [ ]:
brooklyn_catch_basins_gdf = catch_basins_gdf.to_crs('epsg:4326').sjoin(brooklyn_gdf.to_crs('epsg:4326'), how='inner', predicate='within')

In [ ]:
len(brooklyn_catch_basins_gdf)

In [ ]:
_ / 13431

**11%** in Brooklyn

Can save the catch basins.

In [ ]:
brooklyn_catch_basins_gdf.to_parquet('../data/processed/brooklyn/brooklyn-catch-basins.parq')